In [1]:
import requests
import time
import os, json
import csv
import concurrent.futures
# from functools import partial
import pandas as pd

In [2]:
# url = "https://refract.fosfor.com/magiccodermodel/6188a3e2-bc43-444e-bfaa-8159fca71b3a/score"
url = "http://vllm-serving-service:8888/v1/completions"
# lltoken = "eyJhbGciOiJSUzI1NiIsInR5cCIgOiAiSldUIiwia2lkIiA6ICJ4WTdTd3k5UE1xaXRDQmNSMm5qcVl6bmoxS3NqZzV3TmdOV0xDVzdyUkhvIn0.eyJleHAiOjE3MzA4ODM3MzgsImlhdCI6MTY5OTI2MTMzOCwiYXV0aF90aW1lIjoxNjk5MjUyMDQ0LCJqdGkiOiJmN2EzMzQwYy1kNDQwLTRlMzUtYjk2ZS04YzBiMTc0Y2RhODAiLCJpc3MiOiJodHRwczovL3JlZnJhY3QtbG9naW4uZm9zZm9yLmNvbS9hdXRoL3JlYWxtcy9tb3NhaWMiLCJhdWQiOlsibW9zYWljLWdhdGVrZWVwZXIiLCJhY2NvdW50Il0sInN1YiI6IjZjMjU4MWU3LWZmMTItNDljNy04MDJmLWI2ZjQzOWQxZDIwMSIsInR5cCI6IkJlYXJlciIsImF6cCI6Im1vc2FpYy1nYXRla2VlcGVyIiwic2Vzc2lvbl9zdGF0ZSI6IjBhY2Y3YWZhLTVmMzMtNGRhZS05OGM3LTQyZDQwYTdlZTM2NiIsImFsbG93ZWQtb3JpZ2lucyI6WyIqIl0sInJlYWxtX2FjY2VzcyI6eyJyb2xlcyI6WyJNTE9QUyIsImxvbmdfbGl2ZWRfdG9rZW4iLCJzcGVjdHJhLWRldmVsb3BlciIsImRlZmF1bHQtcm9sZXMtbW9zYWljIiwicmVmcmFjdC1kZXZlbG9wZXIiLCJvZmZsaW5lX2FjY2VzcyIsImFkbWluIiwidW1hX2F1dGhvcml6YXRpb24iLCJyZWZyYWN0LWFkbWluIl19LCJyZXNvdXJjZV9hY2Nlc3MiOnsiYWNjb3VudCI6eyJyb2xlcyI6WyJtYW5hZ2UtYWNjb3VudCIsIm1hbmFnZS1hY2NvdW50LWxpbmtzIiwidmlldy1wcm9maWxlIl19fSwic2NvcGUiOiJvcGVuaWQgZW1haWwgcHJvZmlsZSIsInNpZCI6IjBhY2Y3YWZhLTVmMzMtNGRhZS05OGM3LTQyZDQwYTdlZTM2NiIsImVtYWlsX3ZlcmlmaWVkIjp0cnVlLCJuYW1lIjoiUmVmcmFjdCBCRlNJIiwicHJlZmVycmVkX3VzZXJuYW1lIjoicmVmcmFjdC5iZnNpQGZvc2Zvci5jb20iLCJnaXZlbl9uYW1lIjoiUmVmcmFjdCIsImZhbWlseV9uYW1lIjoiQkZTSSIsImVtYWlsIjoicmVmcmFjdC5iZnNpQGZvc2Zvci5jb20ifQ.b6SYLgjo9Veo3GmJ8eZjCTNupQjpfMhzsoXdYjWwRtvRnNjBfx0gOqcugO9OcGn-mm8wwpSGI5uiL30-I6SdWBjsf1ur6GztoX7j-nP_3SrJJn3UhNNqIO8LbsPi5gGRTzWtnfjz92BF1YaCXxQwPY0P_aa8vJ6JxZz5Uctn9aIPIJZZnnjC_GPXtXurmshM_tEN2kwCjhEyr7wYzRqUoMtBGfpLjZREBzgZY-x6JyYiXNtycb1d6PFcCXf7nJVV8ienEC_x7OuciDzfeqd-SQnImvAHH7rqFdi9smBN08AbkDS2uAbMrokHrmbiBpaimrR013VwCWz2KL5QYlWleA"
request_payload_file = '/data/Magicoder_payloads.txt'
report_path = '/data/vLLM_5Concurrency_1gpu.csv'
df = pd.DataFrame(columns=['Prompt', 'Response', 'Response_status', 'Time (seconds)'])

In [3]:
payload = {"prompt": "Problem: Write a function in python to check if two strings are anagrams.Input: Strings s1 and s2.Output: Boolean, indicating whether s1 and s2 are anagrams.", 
           "max_tokens": 200,
           "temperature": 0,
           "model": "/llmmodels/models--ise-uiuc--Magicoder-S-DS-6.7B/snapshots/cff055b1e110cbe75c0c3759bd436299c6d6bb66"}

In [4]:
def read_payloads(file_path):
    with open(file_path, 'r') as file:
        for line in file:
            prompt = json.loads(line.strip())
            yield prompt['payload']

In [5]:
def my_decorator(func):
    def calculate_execution_time(*args, **kwargs):
        global df
        start_time = time.time()
        result, _= func(*args, **kwargs)
        end_time = time.time()
        execution_time = end_time - start_time
        data = {'Prompt':_,'Response':result.text, 'Response_status':result.status_code, 'Time (seconds)':f'{execution_time:.6f}'}
        df = df.append(data, ignore_index=True)
        print(f"Execution time: {execution_time:.6f} seconds \n\n")
        return result
    return calculate_execution_time

In [6]:
def save_report(file_path, df):
    print(df)
    df.to_csv(file_path, index=False)

In [7]:
@my_decorator
def inference(payload_data=None):
    headers = {
        'Content-Type': 'application/json'
    }
    try:
        response = requests.request("POST", url, headers=headers, json=payload_data)
        print(f"Prompt==> {payload_data['prompt']} \n {response.text}")
        return response, payload_data['prompt']
    except Exception as e:
        print(e)
        

In [ ]:
#Model Inference(Request)
if __name__ == "__main__":
    # Using the generator function to read lines 
    prompts = list(read_payloads(request_payload_file))
    
    global df
    for i in range(5):
        print(f"loop {i}+++++++++++++++++++++++++++++")
        with concurrent.futures.ThreadPoolExecutor(max_workers=5) as executor:
            futures = [executor.submit(inference, {**payload, 'prompt': prompt}) for prompt in prompts]
            concurrent.futures.wait(futures)
    save_report(report_path, df)

loop 0+++++++++++++++++++++++++++++
Prompt==> Problem: Write a function in python to reverse a given string.Input: String s.Output: String, the reversed version of s. 
 {"id":"cmpl-7342ae55594643799e58c68b8a50e011","object":"text_completion","created":452771,"model":"/llmmodels/models--ise-uiuc--Magicoder-S-DS-6.7B/snapshots/cff055b1e110cbe75c0c3759bd436299c6d6bb66","choices":[{"index":0,"text":"\n\n\ndef reverse_string(s):\n    return s[::-1]\n\n# Test the function\nprint(reverse_string(\"Hello, World!\"))  # Output: !dlroW ,olleH","logprobs":null,"finish_reason":"stop"}],"usage":{"prompt_tokens":31,"total_tokens":82,"completion_tokens":51}}
Execution time: 2.841400 seconds 


Prompt==> Problem: Write a function in python to check if a given string is a palindrome.Input: String s.Output: Boolean, indicating whether s is a palindrome. 
 {"id":"cmpl-b9a4b3986a8b4d459b73b5f4c5e57adf","object":"text_completion","created":452771,"model":"/llmmodels/models--ise-uiuc--Magicoder-S-DS-6.7B/sna

Prompt==> Problem: Write a function in java to reverse a given string.Input: String s.Output: String, the reversed version of s. 
 {"id":"cmpl-4ca7f2e87660401aa043efe7affc47ce","object":"text_completion","created":452782,"model":"/llmmodels/models--ise-uiuc--Magicoder-S-DS-6.7B/snapshots/cff055b1e110cbe75c0c3759bd436299c6d6bb66","choices":[{"index":0,"text":"\n\n\npublic class Main {\n    public static void main(String[] args) {\n        String s = \"Hello World\";\n        System.out.println(reverseString(s));\n    }\n\n    public static String reverseString(String s) {\n        String reversed = \"\";\n        for (int i = s.length() - 1; i >= 0; i--) {\n            reversed += s.charAt(i);\n        }\n        return reversed;\n    }\n}","logprobs":null,"finish_reason":"stop"}],"usage":{"prompt_tokens":31,"total_tokens":146,"completion_tokens":115}}
Execution time: 6.123981 seconds 


Prompt==> Problem: Write a function in java to calculate the factorial of a given non-negative integ

Prompt==> Problem: Write a function in java to perform binary search on a sorted list.Input: Sorted list arr and target value target.Output: Integer, the index of target in arr (or -1 if not found). 
 {"id":"cmpl-617dcc28b09b4729ba97b1b616407c1f","object":"text_completion","created":452796,"model":"/llmmodels/models--ise-uiuc--Magicoder-S-DS-6.7B/snapshots/cff055b1e110cbe75c0c3759bd436299c6d6bb66","choices":[{"index":0,"text":"\n\n\n```java\npublic class BinarySearch {\n    public static int binarySearch(int[] arr, int target) {\n        int left = 0;\n        int right = arr.length - 1;\n\n        while (left <= right) {\n            int mid = left + (right - left) / 2;\n\n            if (arr[mid] == target) {\n                return mid;\n            } else if (arr[mid] < target) {\n                left = mid + 1;\n            } else {\n                right = mid - 1;\n            }\n        }\n\n        return -1;\n    }\n\n    public static void main(String[] args) {\n        int[

Prompt==> Problem: Write a function in javascript to reverse a given string.Input: String s.Output: String, the reversed version of s. 
 {"id":"cmpl-98c72972e82745348444362a74f97493","object":"text_completion","created":452820,"model":"/llmmodels/models--ise-uiuc--Magicoder-S-DS-6.7B/snapshots/cff055b1e110cbe75c0c3759bd436299c6d6bb66","choices":[{"index":0,"text":"\n\n\nfunction reverseString(s) {\n    return s.split(\"\").reverse().join(\"\");\n}\n\nconsole.log(reverseString(\"Hello World\")); // Output: \"dlroW olleH\"","logprobs":null,"finish_reason":"stop"}],"usage":{"prompt_tokens":31,"total_tokens":82,"completion_tokens":51}}
Execution time: 2.608400 seconds 


Prompt==> Problem: Write a function in javascript to calculate the factorial of a given non-negative integer.Input: Non-negative integer n. Output: Integer, the factorial of n. 
 {"id":"cmpl-85ef32466d1b460782d41b6d7c61b4fb","object":"text_completion","created":452819,"model":"/llmmodels/models--ise-uiuc--Magicoder-S-DS-6.

Prompt==> Problem: Write a function in Ruby to calculate the factorial of a given non-negative integer.Input: Non-negative integer n. Output: Integer, the factorial of n. 
 {"id":"cmpl-eabfe956a1534db6a7dd57b315baed48","object":"text_completion","created":452834,"model":"/llmmodels/models--ise-uiuc--Magicoder-S-DS-6.7B/snapshots/cff055b1e110cbe75c0c3759bd436299c6d6bb66","choices":[{"index":0,"text":"\n\n\ndef factorial(n)\n  if n == 0\n    1\n  else\n    n * factorial(n-1)\n  end\nend\n\nputs factorial(5)","logprobs":null,"finish_reason":"stop"}],"usage":{"prompt_tokens":41,"total_tokens":88,"completion_tokens":47}}
Execution time: 2.498049 seconds 


Prompt==> Problem: Write a function in javascript to generate the nth Fibonacci number.Input: Non-negative integer n.Output: Integer, the nth Fibonacci number. 
 {"id":"cmpl-833a1d9addb744f08b834539e948ab10","object":"text_completion","created":452827,"model":"/llmmodels/models--ise-uiuc--Magicoder-S-DS-6.7B/snapshots/cff055b1e110cbe75c0c

...

Prompt==> Problem: Write a function in jQuery to fetch data from an API and display it on a webpage.Input: API endpoint URL.Output: None (update the webpage with fetched data). 
 {"id":"cmpl-f7690d6665244571ae0075e2b8e9d3aa","object":"text_completion","created":453121,"model":"/llmmodels/models--ise-uiuc--Magicoder-S-DS-6.7B/snapshots/cff055b1e110cbe75c0c3759bd436299c6d6bb66","choices":[{"index":0,"text":"\n\n\nHere is a simple example of how you can fetch data from an API using jQuery and display it on a webpage:\n\n```javascript\n$(document).ready(function() {\n    $.ajax({\n        url: 'https://api.example.com/data', // replace with your API endpoint URL\n        type: 'GET',\n        dataType: 'json',\n        success: function(data) {\n            // Assuming the API returns an array of objects\n            // and each object has a 'name' property\n            $.each(data, function(index, item) {\n                // Append each item's name to the body of the webpage\n            

Prompt==> Problem: Create a function in jQuery to toggle a specific class on an HTML element when a button is clicked.Input: HTML element selector, class name.Output: None (update the class of the HTML element). 
 {"id":"cmpl-29ab77d75e9d4aec8cf34e40a8deb3bc","object":"text_completion","created":453138,"model":"/llmmodels/models--ise-uiuc--Magicoder-S-DS-6.7B/snapshots/cff055b1e110cbe75c0c3759bd436299c6d6bb66","choices":[{"index":0,"text":"\n\nExample:\nHTML:\n<button id=\"toggleButton\">Toggle Class</button>\n<div id=\"myDiv\" class=\"myClass\">Hello World</div>\n\njQuery:\n$(document).ready(function() {\n    $(\"#toggleButton\").click(function() {\n        $(\"#myDiv\").toggleClass(\"myClass\");\n    });\n});\n\nIn this example, when the button with id \"toggleButton\" is clicked, the class \"myClass\" of the div with id \"myDiv\" will be toggled. If the div has the class \"myClass\", it will be removed, and if it doesn't have the class \"myClass\", it will be added.","logprobs":null

Prompt==> Problem: Write a function in python to perform binary search on a sorted list.Input: Sorted list arr and target value target.Output: Integer, the index of target in arr (or -1 if not found). 
 {"id":"cmpl-96bfd6be78b54e49a9cb4dad0983021f","object":"text_completion","created":453153,"model":"/llmmodels/models--ise-uiuc--Magicoder-S-DS-6.7B/snapshots/cff055b1e110cbe75c0c3759bd436299c6d6bb66","choices":[{"index":0,"text":"\n\n\ndef binary_search(arr, target):\n    left, right = 0, len(arr) - 1\n    while left <= right:\n        mid = (left + right) // 2\n        if arr[mid] == target:\n            return mid\n        elif arr[mid] < target:\n            left = mid + 1\n        else:\n            right = mid - 1\n    return -1\n\n# Test the function\narr = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]\ntarget = 5\nprint(binary_search(arr, target))  # Output: 4\n\ntarget = 11\nprint(binary_search(arr, target))  # Output: -1","logprobs":null,"finish_reason":"stop"}],"usage":{"prompt_tokens":47,"

Prompt==> Problem: Write a function in java to generate the nth Fibonacci number.Input: Non-negative integer n.Output: Integer, the nth Fibonacci number. 
 {"id":"cmpl-b382224483064200b5c1086862fd7ac0","object":"text_completion","created":453168,"model":"/llmmodels/models--ise-uiuc--Magicoder-S-DS-6.7B/snapshots/cff055b1e110cbe75c0c3759bd436299c6d6bb66","choices":[{"index":0,"text":"\n\nFibonacci sequence: 0, 1, 1, 2, 3, 5, 8, 13, 21, 34, 55, 89, 144, 233, 377, 610, 987, 1597, 2584, 4181, 6765, 10946, …\n\nHere is a simple Java function to generate the nth Fibonacci number:\n\n```java\npublic class Main {\n    public static void main(String[] args) {\n        System.out.println(fibonacci(10));  // Output: 55\n    }\n\n    public static int fibonacci(int n) {\n        if (n <= 1) {","logprobs":null,"finish_reason":"length"}],"usage":{"prompt_tokens":43,"total_tokens":243,"completion_tokens":200}}
Execution time: 10.536440 seconds 


Prompt==> Problem: Write a function in java to count t

Prompt==> Problem: Write a function in c++ to calculate the sum of the digits of a given positive integer.Input: Positive integer num.Output: Integer, the sum of the digits of num. 
 {"id":"cmpl-b556d742b46b463b8eeee65a122b62fb","object":"text_completion","created":453184,"model":"/llmmodels/models--ise-uiuc--Magicoder-S-DS-6.7B/snapshots/cff055b1e110cbe75c0c3759bd436299c6d6bb66","choices":[{"index":0,"text":"\n\n\nSolution:\n\n```cpp\n#include <iostream>\nusing namespace std;\n\nint sumOfDigits(int num) {\n    int sum = 0;\n    while (num != 0) {\n        sum += num % 10;\n        num /= 10;\n    }\n    return sum;\n}\n\nint main() {\n    int num;\n    cout << \"Enter a positive integer: \";\n    cin >> num;\n    cout << \"Sum of digits: \" << sumOfDigits(num) << endl;\n    return 0;\n}\n```\n\nThis program first asks the user to input a positive integer. It then calls the function `sumOfDigits` with the input number as the argument. The function calculates the sum of the digits by re

In [ ]:
# import aiohttp
# import asyncio
# import time

# start_time = time.time()
# headers={"Content-type":"application/json"} 
# async def main():

#     async with aiohttp.ClientSession(trust_env = True, headers=headers) as session:

# #         for payload in payloads:
# #             url='https://refract.fosfor.com/openchatfinetunedimqlgenerator/d0721c53-c9f5-4d79-a988-2d21df46198f/score' 
#         async with session.post(url, json=payload) as resp:
#                 mql = await resp.json(content_type=None)
#                 print(mql)

# await main()
# print("--- %s seconds ---" % (time.time() - start_time))